In [ ]:
from bs4 import BeautifulSoup
import datetime
import glob 
import json
from openai import OpenAI
import os
import pandas as pd
from pathlib import Path
import re
import requests
from tqdm import tqdm 


In [ ]:
# get the cookie from web browser and paste it in a text file. Should look something like this: 
# kerkspot=<some characters>; ident=<some characters>; s=1
with open("auth_cookie.txt", 'r') as f:
    auth_cookie = f.readline().strip()

In [ ]:
RERUN_GET_MSG_IDS = False
RERUN_SCRAPE = False
RERUN_OPENAI = True

In [ ]:
def get_agenda(year: int, month: int):
    url = f"https://www.immanuelkerkdelft.nl/go/gemeente/agenda/?d={month}|{year}&g="
    headers = {'Cookie': auth_cookie}
    return requests.request("GET", url, headers=headers)

def get_event_ids_month(soup):
        event_ids = []
        all_services = soup.find_all("div", class_="show-info soortID1393")
        for service_event in all_services:
            event_id_text = service_event.get('id')
            event_id_match = re.match(r"eventID(\d+)", event_id_text)
            if event_id_match:
                event_ids.append(event_id_match[1])
        return event_ids

def get_page(id):
    url = f"https://www.immanuelkerkdelft.nl/agenda/{id}/"
    headers = {'Cookie': auth_cookie}
    return requests.request("GET", url, headers=headers)

In [ ]:
if RERUN_GET_MSG_IDS:
    month_from = 1
    year_from = 2023
    month_to = 1
    year_to = 2025

    year_month = []
    for year in range(year_from, year_to+1):
        for month in range(month_from if year == year_from else 1, (month_to if year==year_to else 12)+1):
            year_month.append((year, month))

    for year, month in tqdm(year_month):
        req = get_agenda(year, month)
        if req.status_code == 200:            
            with open(f"./data/agenda_{year}_{month}.html", "w") as outp_file:
                outp_file.write(req.text)
    
    ids_to_search = []
    file_list = sorted(Path("./data").glob("agenda_*"))
    for file in tqdm(file_list):
        with open(file, 'r') as f:
            data = f.read()   
            soup = BeautifulSoup(data, 'html.parser')
            event_ids = get_event_ids_month(soup)
            ids_to_search.extend(event_ids)

    with open('./data/event_ids.json', 'w') as f:
        json.dump(ids_to_search, f)

In [ ]:
if RERUN_SCRAPE:

    with open('./data/event_ids.json', 'r') as f:
        ids_to_search = json.load(f)

    found_ids = []
    for i in tqdm(ids_to_search):
        r = get_page(i)
        if r.status_code == 200:
            found_ids.append(i)
            with open(f"./data/service_{i}.html", 'w', encoding="utf-8") as o:
                o.write(r.text)
    
    print(f"Found ids: {found_ids}")
    print(f"Max: {max(found_ids)}, min: {min(found_ids)}")

In [ ]:
file_list = [f for f in glob.glob("./data/service_*.html")]

len(file_list)

In [ ]:
def get_content(soup):
    text = soup.find(id="agendaContent").get_text(separator='\n')
    cleaned_text = '\n'.join(line for line in text.split('\n') if line.strip())
    return cleaned_text

def get_date(soup):
    month_map = {
        "januari": 1,
        "februari": 2,
        "maart": 3,
        "april": 4,
        "mei": 5,
        "juni": 6,
        "juli": 7,
        "augustus": 8,
        "september": 9,
        "oktober": 10,
        "november": 11,
        "december": 12
    }
    date_line = soup.find("div", {"class": "agendaDatum"}).get_text().strip()
    match = re.match(r"(.*) ([\d\:]+) tot .*", date_line)
    if match:
        day, mth, year = match[1].split()
        return datetime.date(int(year), month_map[mth], int(day))
    return date_line

def filter_content(content: str) -> str:
    """ Remove all lines up to and including the line containing 'liturgie' (case insensitive)
    And remove all lines from the line containing 'Rooster' or 'Vragen'"""
    content_lines = content.split('\n')
    filtered_with_header = []
    filtered_liturgy_only = []
    liturgy_section = False
    for ln in content_lines:
        if any([ln.strip().lower().startswith(keyword) for keyword in 
                  ["rooster", "vragen", "gespreksvragen", "bespreekvragen", "houd me op de hoogte"]]):
                break
        filtered_with_header.append(ln)
        if not liturgy_section:
            liturgy_section = "liturgie" in ln.lower()
        else:
            if "youtube" in ln.lower():
                break
            filtered_liturgy_only.append(ln)
    if not liturgy_section:
        return '\n'.join(filtered_with_header)
    return '\n'.join(filtered_liturgy_only)



In [ ]:
parsed = []

for file in file_list:
    with open(file, 'r', encoding="utf-8") as f:
        data = f.read()
    soup = BeautifulSoup(data, 'html.parser')
    date = get_date(soup)
    content = get_content(soup)
    liturgy = filter_content(content)
    
    parsed.append((date, liturgy, file, content))

In [ ]:
df = pd.DataFrame(parsed, columns=["datum", "liturgie", "file", "content"]).sort_values("file")
df.to_excel("zangdiensten.xlsx")

In [ ]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

system_prompt = """
De gebruiker deelt een beschrijving van een kerkdienst. Vind die liederen, psalmen en gezangen in de Liturgie als een lijst. 
Antwoord alleen met deze lijst. Antwoord altijd in het Nederlands. Antwoord in een consistente opmaak. 
Als je niks vindt, antwoord dan met een lege lijst. Antwoord altijd met alleen een lijst.
"""

example = """Hemelvaart: eens geen volk, nu Gods volk! \n'
De kerkdienst wordt gehouden in de Immanuelkerk en wordt ook uitgezonden via 
Youtube.\n
Klik \n
hier\n
 om de dienst van deze week te bekijken.\n
Liturgie\n
Opening van de dienst: welkom!\n
 Openingspsalm: Psalm 47 (DNP)\n
 Wat is Hemelvaart? Introductie op de viering van Hemelvaart\n
 Lied: Al heeft Hij ons verlaten (NLB 663)\n
Kinderkunst Kidz678\n
Gebed voor de opening van het Woord\n
 Eerste lezing: Lucas 24,36-53\n
 Lied: Thy hand o God has guided (vers 1)\n
 Tweede lezing: 1 Petrus 1,22 - 2,10\n
 Lied: Thy hand o God has guided (vers 3)\n
 Preek\n
 Psalm 118:1,2,6,7 (DNP)\xa0\n
Collecte en voorbeden\n
 Slotlied: NLB 675:1\n
 Zegen (en gezongen amen)\n
"""

example_2 = """
De kerkdienst wordt gehouden in de Immanuelkerk en wordt ook uitgezonden via 
Youtube\n
Klik \n
hier\n
 om de dienst van deze week te bekijken\n
Liturgie\n
Votum en groet – Sela\n
 Welkom\n
 Belijdenis\n
 Lied Pvn 16 (Ik val niet uit zijn hand)\n
 Kort intro\n
 Gebed\n
 Lied Pvn 130 (Uit de diepten)\n
 Bijbellezing Gen. 37 : 2b – 11\n
 Lied OPW 733 (Tienduizend redenen)\n
 Preek\n
 Lied OPW 599 (Kom tot de Vader)\n
 Mededelingen / collecte\n
 Voorbede\n
 Lied OPW 595 (Licht van de wereld)\n
 Zegen\n
Rooster\n
 Houd me op de hoogte\n
\xa0\xa0Delen\n
\xa0\xa0Tweeten\n
\xa0\xa0LinkedIn\n
\xa0\xa0E-mail\n
 Opslaan in mijn agenda\n
 Outlook\n
 Google Kalender\n
 iCalendar
 """

example_response_2 = """\
Votum en groet - Sela
Psalmen voor Nu 16 (Ik val niet uit zijn hand)
Psalmen voor Nu 130 (Uit de diepten)
Opwekking 733 (Tienduizend redenen)
Opwekking 599 (Licht van de wereld)
"""

example_response = """\
Psalm 47 (DNP)
Al heeft Hij ons verlaten (NLB 663)
Thy hand o God has guided (vers 1 en 3)
Psalm 118:1,2,6,7 (DNP)
NLB 675:1
"""

def extract_songs_from_liturgy(text):
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": example},
            {"role": "assistant", "content": example_response},
            {"role": "user", "content": example_2},
            {"role": "assistant", "content": example_response_2},
            {"role": "user", "content": text},
        ],
        model="gpt-3.5-turbo",
    )
    return chat_completion.choices[0].message.content


In [ ]:
if RERUN_OPENAI:
    responses = {}
    for index, row in tqdm(df.iterrows()):
        responses[row.file] = extract_songs_from_liturgy(row.content)

    with open("api_responses.json", 'w') as o:
        o.write(json.dumps(responses))

In [ ]:
with open("api_responses.json", 'r') as f:
    responses = json.loads(f.read())


In [ ]:
ai_summaries = pd.DataFrame([(key, value) for key, value in responses.items()], columns=["file", "songs_ai"])
combined = ai_summaries.merge(df)

In [ ]:
exploded = combined.copy()
exploded["songs_ai"] = exploded.apply(lambda row: row["songs_ai"].split("\n"), axis=1)
exploded = exploded.explode("songs_ai")
exploded = exploded[["file", "songs_ai", "date"]]
exploded

In [ ]:
combined.to_excel("zangdiensten.xlsx")

In [ ]:
exploded.to_excel("liedjes.xlsx")